In [2]:
import pandas as pd
import os

# Load data
df = pd.read_csv('Raw.csv')

# Cek struktur data
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())

Shape: (50000, 11)

Columns: ['Model', 'Year', 'Region', 'Color', 'Fuel_Type', 'Transmission', 'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume', 'Sales_Classification']

First 5 rows:
      Model  Year         Region  Color Fuel_Type Transmission  Engine_Size_L  \
0  5 Series  2016           Asia    Red    Petrol       Manual            3.5   
1        i8  2013  North America    Red    Hybrid    Automatic            1.6   
2  5 Series  2022  North America   Blue    Petrol    Automatic            4.5   
3        X3  2024    Middle East   Blue    Petrol    Automatic            1.7   
4  7 Series  2020  South America  Black    Diesel       Manual            2.1   

   Mileage_KM  Price_USD  Sales_Volume Sales_Classification  
0      151748      98740          8300                 High  
1      121671      79219          3428                  Low  
2       10991     113265          6994                  Low  
3       27255      60971          4047                  Low  
4      12

In [3]:
# Cek masalah data dasar
print("=== DATA QUALITY CHECK ===")
print("Missing values:")
print(df.isnull().sum())
print(f"\nDuplicates: {df.duplicated().sum()}")

# Hapus duplikat
df_clean = df.drop_duplicates()
print(f"Data after removing duplicates: {df_clean.shape}")

# Standardize text columns
df_clean['Model'] = df_clean['Model'].str.upper().str.strip()
df_clean['Region'] = df_clean['Region'].str.title().str.strip()
df_clean['Color'] = df_clean['Color'].str.title().str.strip()
df_clean['Fuel_Type'] = df_clean['Fuel_Type'].str.title().str.strip()
df_clean['Transmission'] = df_clean['Transmission'].str.title().str.strip()

print("\n=== AFTER CLEANING ===")
print("Unique models:", df_clean['Model'].unique())
print("Unique regions:", df_clean['Region'].unique())

=== DATA QUALITY CHECK ===
Missing values:
Model                   0
Year                    0
Region                  0
Color                   0
Fuel_Type               0
Transmission            0
Engine_Size_L           0
Mileage_KM              0
Price_USD               0
Sales_Volume            0
Sales_Classification    0
dtype: int64

Duplicates: 0
Data after removing duplicates: (50000, 11)

=== AFTER CLEANING ===
Unique models: ['5 SERIES' 'I8' 'X3' '7 SERIES' 'M5' '3 SERIES' 'X1' 'M3' 'X5' 'I3' 'X6']
Unique regions: ['Asia' 'North America' 'Middle East' 'South America' 'Europe' 'Africa']


In [4]:
# Buat folder untuk menyimpan hasil
os.makedirs('cleaned_data/by_year', exist_ok=True)
os.makedirs('cleaned_data/by_model', exist_ok=True)

# Pisahkan berdasarkan rentang tahun
print("\n=== SPLITTING BY YEAR ===")

# Definisikan rentang tahun
year_ranges = [
    (2010, 2014, "2010-2014"),
    (2015, 2019, "2015-2019"),
    (2020, 2024, "2020-2024")
]

for start, end, label in year_ranges:
    year_data = df_clean[df_clean['Year'].between(start, end)]
    filename = f"cleaned_data/by_year/cars_{label}.csv"
    year_data.to_csv(filename, index=False)
    print(f"{label}: {len(year_data)} records -> {filename}")

# Simpan semua data yang sudah dibersihkan
df_clean.to_csv("cleaned_data/all_cars_cleaned.csv", index=False)
print(f"\nFull cleaned data: {len(df_clean)} records -> cleaned_data/all_cars_cleaned.csv")


=== SPLITTING BY YEAR ===
2010-2014: 16616 records -> cleaned_data/by_year/cars_2010-2014.csv
2015-2019: 16683 records -> cleaned_data/by_year/cars_2015-2019.csv
2020-2024: 16701 records -> cleaned_data/by_year/cars_2020-2024.csv

Full cleaned data: 50000 records -> cleaned_data/all_cars_cleaned.csv


In [5]:
print("\n=== SPLITTING BY MODEL TYPE ===")

# Group model berdasarkan seri
model_categories = {
    'Regular_Series': ['3 SERIES', '5 SERIES', '7 SERIES'],
    'X_Series': ['X1', 'X3', 'X5', 'X6'],
    'M_Performance': ['M3', 'M5'],
    'I_Series': ['I3', 'I8']
}

for category, models in model_categories.items():
    category_data = df_clean[df_clean['Model'].isin(models)]
    filename = f"cleaned_data/by_model/{category}.csv"
    category_data.to_csv(filename, index=False)
    print(f"{category}: {len(category_data)} records -> {filename}")


=== SPLITTING BY MODEL TYPE ===
Regular_Series: 13853 records -> cleaned_data/by_model/Regular_Series.csv
X_Series: 18032 records -> cleaned_data/by_model/X_Series.csv
M_Performance: 8891 records -> cleaned_data/by_model/M_Performance.csv
I_Series: 9224 records -> cleaned_data/by_model/I_Series.csv


In [6]:
print("\n=== SUMMARY ===")
print(f"Total records setelah cleaning: {len(df_clean)}")

# Analisis per tahun
print("\nRecords per year:")
print(df_clean['Year'].value_counts().sort_index())

# Analisis per model
print("\nRecords per model:")
print(df_clean['Model'].value_counts())

# Analisis per region
print("\nRecords per region:")
print(df_clean['Region'].value_counts())


=== SUMMARY ===
Total records setelah cleaning: 50000

Records per year:
Year
2010    3330
2011    3278
2012    3332
2013    3326
2014    3350
2015    3358
2016    3365
2017    3290
2018    3278
2019    3392
2020    3205
2021    3372
2022    3478
2023    3219
2024    3427
Name: count, dtype: int64

Records per model:
Model
7 SERIES    4666
I3          4618
I8          4606
3 SERIES    4595
5 SERIES    4592
X1          4570
X3          4497
X5          4487
M5          4478
X6          4478
M3          4413
Name: count, dtype: int64

Records per region:
Region
Asia             8454
Middle East      8373
North America    8335
Europe           8334
Africa           8253
South America    8251
Name: count, dtype: int64


In [7]:
# Identifikasi outlier pada harga
print("\n=== PRICE ANALYSIS ===")
print(df_clean['Price_USD'].describe())

# Identifikasi outlier pada mileage
print("\n=== MILEAGE ANALYSIS ===")
print(df_clean['Mileage_KM'].describe())

# Filter outlier jika diperlukan
Q1 = df_clean['Price_USD'].quantile(0.25)
Q3 = df_clean['Price_USD'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_outliers = df_clean[
    (df_clean['Price_USD'] >= lower_bound) &
    (df_clean['Price_USD'] <= upper_bound)
]

print(f"Data setelah remove price outliers: {len(df_no_outliers)} records")
df_no_outliers.to_csv("cleaned_data/cars_no_outliers.csv", index=False)


=== PRICE ANALYSIS ===
count     50000.000000
mean      75034.600900
std       25998.248882
min       30000.000000
25%       52434.750000
50%       75011.500000
75%       97628.250000
max      119998.000000
Name: Price_USD, dtype: float64

=== MILEAGE ANALYSIS ===
count     50000.000000
mean     100307.203140
std       57941.509344
min           3.000000
25%       50178.000000
50%      100388.500000
75%      150630.250000
max      199996.000000
Name: Mileage_KM, dtype: float64
Data setelah remove price outliers: 50000 records


In [8]:
# Buat file summary
with open("cleaned_data/cleaning_report.txt", "w") as f:
    f.write("DATA CLEANING REPORT\n")
    f.write("==================\n\n")
    f.write(f"Original records: {len(df)}\n")
    f.write(f"After cleaning: {len(df_clean)}\n")
    f.write(f"Duplicates removed: {len(df) - len(df_clean)}\n\n")

    f.write("Files created:\n")
    f.write("- all_cars_cleaned.csv (full dataset)\n")
    f.write("- cars_no_outliers.csv (without extreme prices)\n")
    f.write("- by_year/ (3 files by year ranges)\n")
    f.write("- by_model/ (4 files by model categories)\n")

print("\n✅ Cleaning completed! Check 'cleaned_data' folder.")


✅ Cleaning completed! Check 'cleaned_data' folder.
